In [1]:
import pandas as pd
import requests
import csv
from bs4 import BeautifulSoup

In [2]:
# Get url
url = 'https://www.startbase.com/organization/creditshelf/'

response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

In [3]:
# Name, logo and teaser
title_card = soup.find('div', 'flex-grow-1 mx-lg-4')
startup_name = title_card.h1.get('data-name')
logo = title_card.h1.get('data-icon')
teaser = title_card.find('div', 'sb-teaser').text.strip()

In [4]:
# Body info
body_card = soup.find('ul', 'unformated row justify-content-start mb-0')
info_cards = body_card.find_all('li')
startup_info = []; labels_info = []

for count, info in enumerate(info_cards):
    labels_info.append(info.text.strip().split(':')[0])
    if count != 9: #10th item is links to social media
        info_piece = info.text.strip()
        # text cleaning
        if '\n ' in info_piece:
            info_piece = info_piece.split('\n ')[1]
        else:
            info_piece = info_piece.split(': ')[1]
        
        if info_piece[0] == ' ':
            info_piece = info_piece[1::]   
    else: #social media
        media_links = []
        for media in info.find_all('a'):
            media_links.append(media.get('href'))
        info_piece = media_links

    startup_info.append(info_piece)

In [5]:
# About info
about_company = soup.find('div', {'class':'sb-card mb-4'}, id='company').p.text

# Team info
member_cards =  soup.find('div', {'class':'sb-card mb-4'}, id='team')
# Get names
member_names = soup.find_all('span',{'class':'name'})
names = [];
for name in member_names:
    names.append(name.text)
# Get labels
member_labels = soup.find_all('span',{'class':'label'})
labels = [];
for label in member_labels:
    labels.append(label.text) 
    
team_members = zip(names, labels)

In [6]:
# Pool data in table
info_data = [startup_name] + [logo] + [teaser] + startup_info + [about_company] + [team_members]
info_labels = ['Name'] + ['Logo'] + ['Teaser'] + labels_info + ['About'] + ['Members']

company_data = dict(zip(info_labels, info_data))

startbase = pd.DataFrame([company_data])
print(startbase)

          Name                                               Logo  \
0  creditshelf  https://cdn.startbase.com/images/1a561f36f4828...   

                                     Teaser               Sector  \
0  Creditshelf - we redesign SME financing!  Finance & Insurance   

  Company Stage Business Model Customer Focus Founding Year        Team Size  \
0  Growth Stage          Other            B2B          2014  11-50 Employees   

  Handelsregister    Headquarter Official Website  \
0           Found  Frankfurt, DE  creditshelf.com   

                                     On Social Media    Legal Entity  \
0  [https://twitter.com/creditshelf, https://www....  creditshelf AG   

                                               About  \
0  creditshelf ist der Finanzierungspartner für m...   

                              Members  
0  <zip object at 0x0000015040CB31C8>  
